# 모든 속보 뉴스에 일별 레이블(_il_label) 달기
- 3 종목:cluster_8:034950, cluster_7:005930, cluster_2:328380
- 일변동률 레이블 가져오기
- 뉴스에 레이블 달기
* 뉴스: 20230101~20240216
* 종목034950: 20230102~20240216


In [ ]:
# 코랩에서 실행시 pyMySQL을 먼저 설치해야 한다
# !pip install JPype1
# !pip install konlpy
# !pip install pyMySQL
# !pip install -U finance-datareader

In [ ]:
#-- Import packages --
import os
import numpy as np
import pandas as pd
#import FinanceDataReader as fdr
import sqlite3
from sqlalchemy import create_engine
#import konlpy
#from konlpy.tag import Okt
import pymysql
from os import replace
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from datetime import datetime
from sqlalchemy import create_engine
import ast
from os import replace
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
import re
import string
import itertools
import warnings
warnings.filterwarnings('ignore')
#-- Import packages :END --

* 한 종목의일 레이블 DB에서 가져오기

In [ ]:
# --get_one_stock_il_label(scode)------------------------------
# 한 종목 일별 레이블 가져오기
def get_one_stock_il_label(scode):
  # DB 연결 준비
  conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
  # DB에서 종목 데이터  추출위한 sql 준비
  sql = f"SELECT * FROM A{scode}_il_rate_label"
  # DB 검색결과를 dataframe에 저장
  il_label_df = pd.read_sql_query(sql, conn)
  # 실행확인을 위한 화면 출력
  #print(il_label_df.head())
  # DB close
  conn.close()
  # df return
  return il_label_df
# --get_one_stock_il_label(scode) :END------------------------

##* 특정 날짜 하루 뉴스 속보 DB에서 읽어오기
- 최초 실행 함수와 그 이후 다른 함수 호출
- get_ondeday_flattened_sokbo(ddate)
- get_flattened_labeled_sokbo(ddate)
##* 결과를 DB 에 저장
- save_sokbo_code_il_label(df_work_result,ddate)

In [ ]:
#-- get_ondeday_flattened_sokbo()--------------------------------
# 당일 속보 DB에서 읽어오기
def get_ondeday_flattened_sokbo(ddate):
  # DB 연결 준비
  conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="비밀번호", db="DB이름", charset = 'utf8')
  # DB에서 종목 데이터  추출위한 sql 준비
  sql = f"SELECT * FROM {ddate}_sokbo_flatten"
  # DB 검색결과를 dataframe에 저장
  oneday_flattened_sokbo_df = pd.read_sql_query(sql, conn)
  # 실행확인을 위한 화면 출력
  #print(oneday_flattened_sokbo_df.head())
  # DB close
  conn.close()
  # df 반환
  return oneday_flattened_sokbo_df
#-- get_ondeday_flattened_sokbo(): End ---------------------------

#-- get_flattened_labeled_sokbo()---------------------------------
# 당일 속보 DB에서 읽어오기
def get_flattened_labeled_sokbo(ddate):
  # DB 연결 준비
  conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="비밀번호", db="DB이름", charset = 'utf8')
  # DB에서 종목 데이터  추출위한 sql 준비
  sql = f"SELECT * FROM {ddate}_sokbo_flatten_label"
  # DB 검색결과를 dataframe에 저장
  oneday_flattened_labeled_sokbo_df = pd.read_sql_query(sql, conn)
  # 실행확인을 위한 화면 출력
  #print(oneday_flattened_labeled_sokbo_df.head())
  # DB close
  conn.close()
  # df 반환
  return oneday_flattened_labeled_sokbo_df
#-- get_flattened_labeled_sokbo(): End ---------------------------

# -- save_sokbo_code_il_label() ----------------------------------
#  flatten된 속보에 레이블 붙여서 DB에 저장하기 
def save_sokbo_code_il_label(df_work_result,ddate):
 # DB 연결 준비
  conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="비밀번호", db="DB이름", charset = 'utf8')
   # 데이터프레임을 DB로 저장, 이미 있으면 덮어쓰기(replace)
  df_work_result.to_sql(name=ddate+'_sokbo_flatten_label', con=conn,if_exists='replace',index=False)
  # DB close
  conn.dispose()
  # 실행확인을 위한 화면 출력
  #print(ddate)
# -- save_sokbo_code_il_label() :End------------------------------

## * 이전 시점p_time부터 당일c_time까지 모든 뉴스에 특정종목 당일의 il_label 넣기
- 최초 실행: NEED_to_execute_1st_part(stock_code)
- 그 이후 실행: set_il_label_with_stock_code(stock_code)
* 뉴스: 20230101~20240216
* 종목034950: 20230102~20240216
* 종목005930: 20230102~20240216
* 종목328380: 20230102~20240216

In [ ]:
#-- set_il_label_with_stock_code() ---------------------------------
def set_il_label_with_stock_code(stock_code):
  # p_time 초기화:20230101
  p_time = pd.to_datetime('20230101').strftime('%Y%m%d') 
  # 당일  초기화
  c_time = p_time                     

  # 한 종목 레이블 가져오기
  il_label_stock_df = get_one_stock_il_label(stock_code)

  #종목 테이블에서 날짜와 레이블 가져오기
  for A,B in il_label_stock_df[['date','il_label']].iterrows():
    # c_time: 당일 날짜
    c_time = B.date
    # 실행 확인을 위한 화면 출력
    print ('past:',p_time,'today:',c_time, 'today label:',B.il_label)
    # 전일p_time부터 당일c_time까지 뉴스 가져다가 레이블 넣기
    while pd.to_datetime(p_time) <= pd.to_datetime(c_time):
      # 실행 확인을 위한 화면 출력
      print('*- p_time',p_time,'c_time', c_time)
      # 이전 날짜의 속보 읽어오기
      temp_df = get_flattened_labeled_sokbo(p_time)
      # 레이블 넣기
      temp_df[stock_code+'_il'] = B.il_label
      # 결과를 DB에 저장하기
      save_sokbo_code_il_label(temp_df,p_time)
      # 실행 확인을 위한 화면 출력
      #print(temp_df.head(3))
      # 이전 날짜 p_time 하루 증가
      p_time = pd.to_datetime(p_time)+pd.Timedelta(days=1)
      # YYYYMMDD 형식의 문자열로 변환
      p_time = p_time.strftime('%Y%m%d')
      # 실행 확인을 위한 화면 출력
      print('*-',p_time, c_time)
#-- set_il_label_with_stock_code():End -----------------------------

#-- ~flatten_label 테이블 없을 경우 최초 실행시키기--------------------
#-- NEED_to_execute_1st_part()--------------------------------------
def NEED_to_execute_1st_part(stock_code):
  # p_time 초기화:20230101
  p_time = pd.to_datetime('20230101').strftime('%Y%m%d') # past
  # 당일 초기화
  c_time = p_time                     

  # 한 종목 레이블 가져오기
  il_label_stock_df = get_one_stock_il_label(stock_code)

  # 종목 테이블에서 날자와 레이블 가져오기
  for A,B in il_label_stock_df[['date','il_label']].iterrows():
    # c_time: 당일 날짜
    c_time = B.date
    # 실행 확인을 위한 화면 출력
    print ('past:',p_time,'today:',c_time, 'today label:',B.il_label)
    # 전일p_time부터 당일c_time까지 뉴스 가져다가 레이블 넣기
    while pd.to_datetime(p_time) <= pd.to_datetime(c_time):
      # 실행 확인을 위한 화면 출력
      print('*- p_time',p_time,'c_time', c_time)
      # 해당 날짜 속보 읽어오기
      temp_df = get_ondeday_flattened_sokbo(p_time)
      # 일별 레이블 넣기
      temp_df[stock_code+'_il'] = B.il_label
      # 결과를 Db에 저장하기
      save_sokbo_code_il_label(temp_df,p_time)
      # 실행 확인을 위한 화면 출력
      #print(temp_df.head(3))
      # 날짜를 하루 증가시키기
      p_time = pd.to_datetime(p_time)+pd.Timedelta(days=1)
      # YYYYMMDD 형식의 문자열로 변환
      p_time = p_time.strftime('%Y%m%d')
      # 실행 확인을 위한 화면 출력
      print('*-',p_time, c_time)
#-- NEED_to_execute_1st_part(): End ----------------------------------

In [ ]:
#=========================================================
# 메인 함수 실행
# 최초 실행: _flatten_label 테이블이 없을 경우 먼저 실행
#---------------------------------------------------------
# cluster_8: '034950'
NEED_to_execute_1st_part('034950')
#=========================================================

In [ ]:
#=======================================================
# 메인 함수 실행
# ~_flatten_label 테이블이 있는 경우 실행
#-------------------------------------------------------
# cluster_7: '005930'
#set_il_label_with_stock_code('005930')
# cluster_2: '328380'
set_il_label_with_stock_code('328380')
#=======================================================